# Exploratory Notebook

ALL group members

## 1. Business Problem and Understanding

**Stakeholders:**

[business problem and background]

**Preliminary Questions include:**

-
-
- 

Databases:

**The goal:**

In [2]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import sqlite3

%matplotlib inline


In [3]:
# Load in csv & tsv data

# tsv's are tabbular and take '\t' as the delimiter
# 'rt.reviews.tsv' did not take default uft-8 decoding

df_gross = pd.read_csv('Data/bom.movie_gross.csv')
df_info = pd.read_csv('Data/rt.movie_info.tsv', delimiter='\t')
df_rev = pd.read_csv('Data/rt.reviews.tsv', delimiter='\t', encoding='latin-1')
df_db = pd.read_csv('Data/tmdb.movies.csv', index_col=0)
df_budg = pd.read_csv('Data/tn.movie_budgets.csv')

In [4]:
# create sql connection to database
conn = sqlite3.connect('Data/im.db')
cur = conn.cursor()

## 2. Dataframe Exploration

In [5]:
# Join movie_basics and movie_ratings & create df

q = ''' SELECT *
FROM movie_basics
JOIN movie_ratings
USING (movie_id)
'''

df_basics = pd.read_sql(q, conn)

In [6]:
# Join persons with directors
q = ''' SELECT *
FROM persons
JOIN directors
USING (person_id)
'''

df_directors = pd.read_sql(q, conn)

In [7]:
# Join persons with writers
q = ''' SELECT *
FROM persons
JOIN writers
USING (person_id)
'''

df_writers = pd.read_sql(q, conn)

In [8]:
# Join persons with known_for
q = ''' SELECT *
FROM persons
JOIN known_for
USING (person_id)
'''

df_known_for = pd.read_sql(q, conn)

In [9]:
# Create df for movie_akas

q = '''
SELECT *
FROM movie_akas
'''
df_akas = pd.read_sql(q,conn)

In [10]:
# Create df for principals

q = '''
SELECT *
FROM principals
'''
df_principals = pd.read_sql(q,conn)

In [11]:
# Create function to get df summary
def df_summary(df):
    print(df.head(3))
    print(df.info())
    print(df.isna().sum())
    print(df.isna().sum()/len(df)*100)

#### `df_basics`

- Drop nas
- Lowercase object columns

In [11]:
df_summary(df_basics)

    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   

   start_year  runtime_minutes              genres  averagerating  numvotes  
0        2013            175.0  Action,Crime,Drama            7.0        77  
1        2019            114.0     Biography,Drama            7.2        43  
2        2018            122.0               Drama            6.9      4517  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         73856 non-null  object 
 1   primary_title    73856 non-null  object 
 2   original_title   73856 non-null  object 
 3   start_year       73856 non-null  int

#### `df_akas`

- Can use to compare language to gross amount

In [12]:
df_summary(df_akas)

    movie_id  ordering                                    title region  \
0  tt0369610        10                            Джурасик свят     BG   
1  tt0369610        11                        Jurashikku warudo     JP   
2  tt0369610        12  Jurassic World: O Mundo dos Dinossauros     BR   

  language        types attributes  is_original_title  
0       bg         None       None                0.0  
1     None  imdbDisplay       None                0.0  
2     None  imdbDisplay       None                0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              16

#### `df_directors`, `df_writers`, `df_known_for`

- Without gender and other socio cultural details, not sure this information can be used.

In [13]:
df_summary(df_directors)
print('___________________________________________________________________________')
df_summary(df_writers)
print('___________________________________________________________________________')
df_summary(df_known_for)

   person_id    primary_name  birth_year  death_year  \
0  nm0062879  Ruel S. Bayani         NaN         NaN   
1  nm0062879  Ruel S. Bayani         NaN         NaN   
2  nm0062879  Ruel S. Bayani         NaN         NaN   

                          primary_profession   movie_id  
0  director,production_manager,miscellaneous  tt1592569  
1  director,production_manager,miscellaneous  tt1592569  
2  director,production_manager,miscellaneous  tt1592569  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291171 entries, 0 to 291170
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           291171 non-null  object 
 1   primary_name        291171 non-null  object 
 2   birth_year          68608 non-null   float64
 3   death_year          1738 non-null    float64
 4   primary_profession  290187 non-null  object 
 5   movie_id            291171 non-null  object 
dtypes: float64(2), object(4)
mem

#### `df_principals`

- Not sure if we will use this table
- If we do, we may be able to fill in from other tables

In [14]:
df_summary(df_principals)

    movie_id  ordering  person_id  category       job   characters
0  tt0111414         1  nm0246005     actor      None  ["The Man"]
1  tt0111414         2  nm0398271  director      None         None
2  tt0111414         3  nm3739909  producer  producer         None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028186 entries, 0 to 1028185
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   movie_id    1028186 non-null  object
 1   ordering    1028186 non-null  int64 
 2   person_id   1028186 non-null  object
 3   category    1028186 non-null  object
 4   job         177684 non-null   object
 5   characters  393360 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.1+ MB
None
movie_id           0
ordering           0
person_id          0
category           0
job           850502
characters    634826
dtype: int64
movie_id       0.000000
ordering       0.000000
person_id      0.000000
category       0.

#### `df_gross`

- `df_budg` has more complete info on gross numbers. May not use.

In [157]:
df_summary(df_gross)

                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype:

#### `df_info`

- Too many missing values. May not use.

In [79]:
df_summary(df_info)

   id                                           synopsis rating  \
0   1  This gritty, fast-paced, and innovative police...      R   
1   3  New York City, not-too-distant-future: Eric Pa...      R   
2   5  Illeana Douglas delivers a superb performance ...      R   

                                 genre          director  \
0  Action and Adventure|Classics|Drama  William Friedkin   
1    Drama|Science Fiction and Fantasy  David Cronenberg   
2    Drama|Musical and Performing Arts    Allison Anders   

                         writer  theater_date      dvd_date currency  \
0                Ernest Tidyman   Oct 9, 1971  Sep 25, 2001      NaN   
1  David Cronenberg|Don DeLillo  Aug 17, 2012   Jan 1, 2013        $   
2                Allison Anders  Sep 13, 1996  Apr 18, 2000      NaN   

  box_office      runtime             studio  
0        NaN  104 minutes                NaN  
1    600,000  108 minutes  Entertainment One  
2        NaN  116 minutes                NaN  
<class 'panda

#### `df_rev`

- Not sure if there is enough significant information.

In [17]:
df_summary(df_rev)

   id                                             review rating   fresh  \
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   

           critic  top_critic         publisher               date  
0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0           io9.com       May 23, 2018  
2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  in

#### `df_db`

- No nulls
- Create new column `release_year` from `release_date`

In [14]:
df_summary(df_db)

             genre_ids     id original_language  \
0      [12, 14, 10751]  12444                en   
1  [14, 12, 16, 10751]  10191                en   
2        [12, 28, 878]  10138                en   

                                 original_title  popularity release_date  \
0  Harry Potter and the Deathly Hallows: Part 1      33.533   2010-11-19   
1                      How to Train Your Dragon      28.734   2010-03-26   
2                                    Iron Man 2      28.515   2010-05-07   

                                          title  vote_average  vote_count  
0  Harry Potter and the Deathly Hallows: Part 1           7.7       10788  
1                      How to Train Your Dragon           7.7        7610  
2                                    Iron Man 2           6.8       12368  
<class 'pandas.core.frame.DataFrame'>
Index: 26517 entries, 0 to 26516
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             -----------

#### `df_budg`

- No nulls
- 'movie' = 'title'
- Adjust 'release_date' if using for analysis
- Adjust 'gross' columns to int

In [15]:
df_summary(df_budg)

   id  release_date                                        movie  \
0   1  Dec 18, 2009                                       Avatar   
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides   
2   3   Jun 7, 2019                                 Dark Phoenix   

  production_budget domestic_gross worldwide_gross  
0      $425,000,000   $760,507,625  $2,776,345,279  
1      $410,600,000   $241,063,875  $1,045,663,875  
2      $350,000,000    $42,762,350    $149,762,350  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memor

## 3. Data Cleaning & Preprocessing

In [12]:
# Lowercase function
def lowercase(df):
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].str.lower()
    return df

#### Cleaning df_basics

In [44]:
# Make cleaning copy
df_basics_c = df_basics.copy()

# Address lowercase & nas
df_basics_c.dropna(inplace=True)
lowercase(df_basics_c)

# Create primary_genre column
df_basics_c['primary_genre'] = df_basics_c.genres.apply(lambda x: x.split(',')[0])

df_basics_c.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,primary_genre
0,tt0063540,sunghursh,sunghursh,2013,175.0,"action,crime,drama",7.0,77,action
1,tt0066787,one day before the rainy season,ashad ka ek din,2019,114.0,"biography,drama",7.2,43,biography
2,tt0069049,the other side of the wind,the other side of the wind,2018,122.0,drama,6.9,4517,drama
4,tt0100275,the wandering soap opera,la telenovela errante,2017,80.0,"comedy,drama,fantasy",6.5,119,comedy
6,tt0137204,joe finds grace,joe finds grace,2017,83.0,"adventure,animation,comedy",8.1,263,adventure


It looks like this dataset is missing 10 years of information from the timeline we are looking at based on the release year in other databases which maxes out at 2020 and the information found about the newest era being post millenium.

**Source:** <a href = "http://www.historyoffilm.net/movie-eras/history-of-cinema/#:~:text=With%20over%20100%20years%20of,every%20decade%20of%20its%20history.">Movie Eras - History of Cinema and the First Film</a>

In [59]:
print("release_year range:", df_basics_c.start_year.min(), df_basics_c.start_year.max())

# Filter out years before 2000
df_basics_c = df_basics_c[df_basics_c['start_year']>1999]
print(df_basics_c.shape)

release_year range: 2010 2019
(65720, 9)


#### Cleaning df_db

In [62]:
# Create a cleaning copy
df_db_c = df_db.copy()

# Address lowercase
lowercase(df_db_c).head(10)
    
# create release_year column and convert to int
df_db_c['release_year'] = df_db_c['release_date'].apply(lambda x: int(x.split('-')[0]))

# strip genre_ids of []
df_db_c['genre_ids'] = df_db_c['genre_ids'].apply(lambda x: x.strip('[]').strip(""))

# extract primary genre_id & create a new column 'primary_genre'
df_db_c['primary_genre'] = df_db_c['genre_ids'].apply(lambda x: x.split(',')[0])


data_text = '''
Action          28
Adventure       12
Animation       16
Comedy          35
Crime           80
Documentary     99
Drama           18
Family          10751
Fantasy         14
History         36
Horror          27
Music           10402
Mystery         9648
Romance         10749
Science Fiction 878
TV Movie        10770
Thriller        53
War             10752
Western         37
'''

# Convert the text into a list of tuples
genre_list = [tuple(line.split(maxsplit=1)) for line in data_text.strip().split('\n')]

# Convert the list of into a dictionary
genre_dict = dict(genre_list)

# Iterate over each genre_ids to replace with genre name bsed on genre_dict
for k, v in genre_dict.items():
    df_db_c.loc[df_db_c['primary_genre'].str.contains(v), 'primary_genre'] = k.lower()

df_db_c.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,release_year,primary_genre
0,"12, 14, 10751",12444,en,harry potter and the deathly hallows: part 1,33.533,2010-11-19,harry potter and the deathly hallows: part 1,7.7,10788,2010,adventure
1,"14, 12, 16, 10751",10191,en,how to train your dragon,28.734,2010-03-26,how to train your dragon,7.7,7610,2010,fantasy
2,"12, 28, 878",10138,en,iron man 2,28.515,2010-05-07,iron man 2,6.8,12368,2010,adventure
3,"16, 35, 10751",862,en,toy story,28.005,1995-11-22,toy story,7.9,10174,1995,animation
4,"28, 878, 12",27205,en,inception,27.920,2010-07-16,inception,8.3,22186,2010,action


In [86]:
# Replace whitespace with None to identify nans
df_db_c['genre_ids'] = df_db_c['genre_ids'].replace(r'^\s*$', np.nan, regex=True)

# ~9% of genre_ids are missing from the data
df_db_c.isna().sum()/len(df_db_c)*100

df_db_c['genre_ids'].dropna(inplace=True)

Noticed there were empty strings not registered as None and therefore were not coming up as nans. After transforming them to proper format, only ~9% of data was missing so it was appropriate to drop those rows.

In [56]:
print("release_year range:", df_db_c.release_year.min(), df_db_c.release_year.max())

# Filter out years before 2000
df_db_c = df_db_c[df_db_c['release_year']>1999]
print(df_db_c.shape)

release_year range: 2000 2020
(26398, 11)


In [163]:
# Explore columns

print("vote_average range:", df_db.vote_average.values.min(), df_db.vote_average.values.max())
print("popularity range:", df_db.popularity.values.min(), df_db.popularity.values.max())
print("release_year range:", df_db.release_year.min(), df_db.release_year.max())
print("top_10_languages:", df_db.original_language.value_counts().head(10))

vote_average range: 0.0 10.0
popularity range: 0.6 80.773
release_year range: 1930 2020
top_10_languages: original_language
en    23291
fr      507
es      455
ru      298
ja      265
de      237
zh      177
hi      172
it      123
pt       96
Name: count, dtype: int64


It looks like `vote_average` is out of 10 so can be considered a 'rating' potentially. It's possible `popularity` is out of 100, though the max is 80.773. The max year is 2020, which may limit other databases we use if we want all years to align

#### Cleaning df_budg

In [ ]:
Grossing

'genres' in df_basics (movie_ratings & movie_basics combined)
'domestic_gross', 'foreign_gross', or 'worldwide_gross
'language' in df_db (tmdb.movies.csv)
Viewability

'averagerating' in df_basics
'genres' in df_basics
'popularity' & 'vote_average' in df_db
'original_language' in df_db

In [48]:
# Create cleaning copy
df_budg_c = df_budg.copy()

# Address lowercase 
lowercase(df_budg_c).head(10)

# Create release_year as an int
df_budg_c['release_year'] = df_budg_c['release_date'].apply(lambda x: int(x.split(',')[1]))

# Convert dollar columns to int
df_budg_c['production_budget'] = df_budg_c['production_budget'].str.replace(',', '').str.replace('$', '').astype(int)
df_budg_c['domestic_gross'] = df_budg_c['domestic_gross'].str.replace(',', '').str.replace('$', '').astype(int)
df_budg_c['worldwide_gross'] = df_budg_c['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(int)

# Create foreign_gross column
df_budg_c['foreign_gross'] = df_budg_c.worldwide_gross - df_budg_c.domestic_gross

df_budg_c.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,foreign_gross
0,1,"dec 18, 2009",avatar,425000000,760507625,2776345279,2009,2015837654
1,2,"may 20, 2011",pirates of the caribbean: on stranger tides,410600000,241063875,1045663875,2011,804600000
2,3,"jun 7, 2019",dark phoenix,350000000,42762350,149762350,2019,107000000
3,4,"may 1, 2015",avengers: age of ultron,330600000,459005868,1403013963,2015,944008095
4,5,"dec 15, 2017",star wars ep. viii: the last jedi,317000000,620181382,1316721747,2017,696540365


In [53]:
print("release_year range:", df_budg_c.release_year.min(), df_budg_c.release_year.max())

# Filter out years before 2000
df_budg_c = df_budg_c[df_budg_c['release_year']>1999]
print(df_budg_c.shape)

release_year range: 2000 2020
(4387, 8)
